In [4]:

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

Discussing things you care about can be difficult. The threat of abuse and harassment online means that many people stop expressing themselves and give up on seeking different opinions. Platforms struggle to effectively facilitate conversations, leading many communities to limit or completely shut down user comments.

The Conversation AI team, a research initiative founded by Jigsaw and Google (both a part of Alphabet) are working on tools to help improve online conversation. One area of focus is the study of negative online behaviors, like toxic comments (i.e. comments that are rude, disrespectful or otherwise likely to make someone leave a discussion). So far they’ve built a range of publicly available models served through the Perspective API, including toxicity. But the current models still make errors, and they don’t allow users to select which types of toxicity they’re interested in finding (e.g. some platforms may be fine with profanity, but not with other types of toxic content).

In this competition, you’re challenged to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate better than Perspective’s current models. You’ll be using a dataset of comments from Wikipedia’s talk page edits. Improvements to the current model will hopefully help online discussion become more productive and respectful.

we are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:

---toxic
---severe_toxic
---obscene
---threat
---insult
---identity_hate
You must create a model which predicts a probability of each type of toxicity for each comment.



In [5]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [6]:
train = pd.read_csv('/Users/harshitadidwania/Desktop/train.csv').fillna(' ')
test = pd.read_csv('/Users/harshitadidwania/Desktop/test.csv').fillna(' ')

FileNotFoundError: [Errno 2] File b'/Users/harshitadidwania/Desktop/train.csv' does not exist: b'/Users/harshitadidwania/Desktop/train.csv'

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [8]:
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [9]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [10]:
train_word_features

<159571x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3666964 stored elements in Compressed Sparse Row format>

#### char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [17]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [19]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')
    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]
                                    

CV score for class toxic is 0.9692156622415217
CV score for class severe_toxic is 0.987594180521523
CV score for class obscene is 0.9838697844991268
CV score for class threat is 0.9833768552793001
CV score for class insult is 0.977426342272607
CV score for class identity_hate is 0.9739428319445258


In [20]:
print('Total CV score is {}'.format(np.mean(scores)))

Total CV score is 0.9792376094597673


In [28]:
predictions= classifier.predict(test_features)

submission.to_csv('submission.csv', index=False)

In [29]:
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [24]:
pd.read_csv('submission.csv')

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.982227,0.087444,0.956480,0.012208,0.864834,0.088031
1,0000247867823ef7,0.027562,0.005407,0.017160,0.002533,0.016121,0.006816
2,00013b17ad220c46,0.026837,0.005304,0.015823,0.002480,0.013956,0.005710
3,00017563c3f7919a,0.013824,0.003824,0.010003,0.002263,0.010683,0.003246
4,00017695ad8997eb,0.060092,0.004325,0.022015,0.002388,0.021508,0.004857
5,0001ea8717f6de06,0.020061,0.004349,0.011832,0.002964,0.016401,0.004715
6,00024115d4cbde0f,0.016444,0.003041,0.011307,0.001713,0.011709,0.003316
7,000247e83dcc1211,0.246518,0.007243,0.047928,0.003706,0.056010,0.010252
8,00025358d4737918,0.025715,0.003041,0.016134,0.002068,0.012916,0.004360
9,00026d1092fe71cc,0.018059,0.003151,0.012632,0.002093,0.012522,0.003711
